In [ ]:
% cd ~/Master_Data_Science/TFM/2_Collect_data/
% pwd

### Collect codes of public elementary schools in municipality of Madrid.

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Url to perform advanced searches.
url_advsearch = "http://www.madrid.org/wpad_pub/run/j/BusquedaAvanzada.icm"

In [ ]:
# Parameter for searching public primary schools in municipality of Madrid
params = {"titularidadPublica": "S", "cdMuni": "079", "cdNivelEdu": "6545"}

In [ ]:
# Request and parse list of schools.
schools = BeautifulSoup(requests.post(url_advsearch, data = params).content, "lxml")

In [ ]:
# Extract list of school codes.
school_codes = schools.findAll(attrs = {"name": "codCentrosExp", "value": re.compile("^.+$")})[0]["value"]

In [ ]:
# Convert from string to list.
school_codes = school_codes.split(";")

# Check codes.
# 247 it's ok.
print len(school_codes)

In [ ]:
# Save the list of school codes

import pickle

with open("school_codes.txt", "wb") as f:
    pickle.dump(school_codes, f)

In [ ]:
# open list of school codes
with open("school_codes.txt", "rb") as f:
    school_codes = pickle.load(f)

### Extract tables from school cards.

In [ ]:
# The data that we want to obtain from each school is contained in tables,
# whose content is generated on-the-fly through JavaScript code.
# We can't use 'request' library again because only fetch source code of
# the web page but it doesn't run code. So we need to mimic the rendering
# process of a browser. Let's use dryscrape

import dryscrape

In [ ]:
# School card url.
url_schoolcard = "http://www.madrid.org/wpad_pub/run/j/MostrarFichaCentro.icm"

# School code parameter.
school_code_par = "cdCentro="

# List of schools urls
schools_urls = [url_schoolcard + "?" + school_code_par + code for code in school_codes]

In [ ]:
render = dryscrape.Session()
render.visit("http://www.madrid.org/wpad_pub/run/j/MostrarFichaCentro.icm?cdCentro=28063799")
source = render.body()
school_card = BeautifulSoup(source, "lxml")
school_tables = school_card.findAll('table', class_="tablaGraficaDatos")

table = list(school_tables)[1]
table

In [ ]:
type(table)

In [ ]:
pd.read_html(table.prettify())

In [11]:
from bs4 import BeautifulSoup
import pandas as pd
import dryscrape

render = dryscrape.Session()
render.visit("http://www.madrid.org/wpad_pub/run/j/MostrarFichaCentro.icm?cdCentro=28063799")
render.driver.exec_script('document.getElementById("nivEd12.grafica3").click();')
#radiob = render.at_css('#nivEd12\.grafica3')
#radiob = render.at_xpath('//*[@id="nivEd12.grafica3"]')
#radiob.click()
source = render.body()
school_card = BeautifulSoup(source, "lxml")
school_tables = school_card.findAll('table', class_="tablaGraficaDatos")
table = list(school_tables)[1]
pd.read_html(table.prettify())

[  PROCESO DE ADMISIÓN (solicitudes presentadas, admitidas, no admitidas)  \
 0                                        Presentadas                       
 1                                          Admitidas                       
 2                                       No admitidas                       
 
    2012-2013  2013-2014  2014-2015  2015-2016  2016-2017  
 0         20         25         25         20         18  
 1         14         11          7          9         11  
 2          6         14         18         11          7  ]

In [ ]:
session = dryscrape.Session()
session.set_attribute('auto_load_images', False)
session.visit("http://www.madrid.org/wpad_pub/run/j/MostrarFichaCentro.icm?cdCentro=28063799")
radiob = session.at_xpath('//*[(@id = "nivEd12\.grafica3")]')
radiob.click()
source = session.body()
school_card = BeautifulSoup(source, "lxml")
school_tables = school_card.findAll('table', class_="tablaGraficaDatos")
table = list(school_tables)[1]
pd.read_html(table.prettify())